<a href="https://colab.research.google.com/github/furkansaygili6/SchedulerVirtualAssistant/blob/main/planlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers rouge_score
!pip intsall accelerate
!pip install transformers[torch]
!pip install accelerate -U


ERROR: unknown command "intsall" - maybe you meant "install"


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import evaluate
import re

# Veri yolunu güncelleyin
data_path = "/content/drive/MyDrive/planlama/final_daily_life_schedules.csv"

# Veriyi yükleme
df = pd.read_csv(data_path)

# Veri temizliği ve ön işleme fonksiyonu
def preprocess_text(text):
    text = text.lower()  # Küçük harfe çevirme
    text = re.sub(r'\s+', ' ', text)  # Gereksiz boşlukları kaldırma
    text = re.sub(r'\s*,\s*', ', ', text)  # Virgüller arasında doğru boşluklar
    text = text.strip()  # Başındaki ve sonundaki boşlukları kaldırma
    return text

# Giriş ve çıkış verilerini temizleme
df['input_text'] = df['Kullanıcı Girdisi'].apply(preprocess_text)
df['output_text'] = df['Görevler'].apply(preprocess_text)

# Temizlenmiş verileri kontrol etme
print(df.head())

# Dataset oluşturma
from datasets import Dataset
dataset = Dataset.from_pandas(df[['input_text', 'output_text']])



                                   Kullanıcı Girdisi  \
0  15:00 ile 17:00 arasında araştırma yapacağım, ...   
1  12:00 ile 13:00 arasında öğle yemeği yapacağım...   
2  08:00 ile 10:00 arasında ders yapacağım, 12:00...   
3  08:00 ile 10:00 arasında ders yapacağım, 10:00...   
4  08:00 ile 10:00 arasında ders yapacağım, 10:00...   

                                            Görevler  \
0  15:00 - 17:00 araştırma, 17:00 - 18:00 spor, 1...   
1  12:00 - 13:00 öğle yemeği, 13:00 - 15:00 ders,...   
2  08:00 - 10:00 ders, 12:00 - 13:00 öğle yemeği,...   
3  08:00 - 10:00 ders, 10:00 - 12:00 ders, 13:00 ...   
4  08:00 - 10:00 ders, 10:00 - 12:00 ders, 18:00 ...   

                                          input_text  \
0  15:00 ile 17:00 arasında araştırma yapacağım, ...   
1  12:00 ile 13:00 arasında öğle yemeği yapacağım...   
2  08:00 ile 10:00 arasında ders yapacağım, 12:00...   
3  08:00 ile 10:00 arasında ders yapacağım, 10:00...   
4  08:00 ile 10:00 arasında ders yapacağım, 10

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch
from sklearn.metrics import f1_score

# Tokenizer ve Model Yükleme
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Padding token ekleme
tokenizer.pad_token = tokenizer.eos_token

# Verileri tokenize etme fonksiyonu
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=256, truncation=True, padding="max_length")
    outputs = tokenizer(examples['output_text'], max_length=256, truncation=True, padding="max_length")
    inputs["labels"] = outputs["input_ids"]
    return inputs

# Verileri tokenize etme
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "output_text"])

# Eğitim ve test seti oluşturma
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
import pandas as pd
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import evaluate


# Eğitim argümanları
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=100,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
)

# F1 skoru hesaplama fonksiyonu
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).flatten().cpu().numpy()
    labels = torch.tensor(labels).flatten().cpu().numpy()
    f1 = f1_score(labels, predictions, average='weighted')
    return {'f1': f1}

# Trainer tanımlama
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Model eğitimi
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,1.082900,1.055776,0.730492
2,0.996000,0.934297,0.728790
3,0.890400,0.865650,0.731604
4,0.883700,0.825006,0.730708
5,0.800500,0.807281,0.729057
6,0.804100,0.783872,0.729486
7,0.821400,0.758530,0.731915
8,0.789200,0.747024,0.733157
9,0.755400,0.739082,0.731236
10,0.773800,0.738758,0.731106


TrainOutput(global_step=1000, training_loss=0.8865238862037659, metrics={'train_runtime': 435.9592, 'train_samples_per_second': 4.588, 'train_steps_per_second': 2.294, 'total_flos': 261292032000000.0, 'train_loss': 0.8865238862037659, 'epoch': 10.0})

In [ ]:
# Modeli kaydetme
model_save_path = "/content/drive/MyDrive/planlama/2trained_model"

# Modeli ve tokenizer'ı kaydetme
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('/content/drive/MyDrive/planlama/2trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/planlama/2trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/planlama/2trained_model/vocab.json',
 '/content/drive/MyDrive/planlama/2trained_model/merges.txt',
 '/content/drive/MyDrive/planlama/2trained_model/added_tokens.json')

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import re
from sklearn.metrics import f1_score

# Model ve tokenizer yükleme
model_save_path = "/content/drive/MyDrive/planlama/2trained_model"
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_save_path)

# Cihaz ayarları
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def format_output(output_text):
    formatted_output = re.sub(r'(\d{2}:\d{2}) ile (\d{2}:\d{2}) arasında', r'\1 - \2', output_text)
    formatted_output = re.sub(r'yapacağım', '', formatted_output)
    formatted_output = formatted_output.replace(',', '')
    return formatted_output.strip()

# Modelden çıktı üretme fonksiyonu
def generate_output(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True, padding="max_length")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return format_output(output_text.strip())

# F1 skoru hesaplama fonksiyonu
def calculate_f1_score(true_output, predicted_output):
    true_output_tokens = true_output.split()
    predicted_output_tokens = predicted_output.split()

    # Uzunlukları eşitlemek için padding yapma
    max_len = max(len(true_output_tokens), len(predicted_output_tokens))
    true_output_tokens += [''] * (max_len - len(true_output_tokens))
    predicted_output_tokens += [''] * (max_len - len(predicted_output_tokens))

    return f1_score(true_output_tokens, predicted_output_tokens, average='weighted')

# Test girişi
input_text = "09:00 ile 10:00 arasında toplantı yapacağım, 12:00 ile 13:00 arasında öğle yemeği yapacağım, 16:00 ile 17:00 arasında arkadaş buluşması yapacağım, 17:00 ile 19:00 arasında yemek randevusu yapacağım, 21:00 ile 22:00 arasında kitap okuma yapacağım"
true_output = "09:00 - 10:00 toplantı, 12:00 - 13:00 öğle yemeği, 16:00 - 17:00 arkadaş buluşması, 17:00 - 19:00 yemek randevusu, 21:00 - 22:00 kitap okuma"

predicted_output = generate_output(input_text)

# F1 skoru hesaplama
f1 = calculate_f1_score(true_output, predicted_output)

print("Giriş:", input_text)
print("Beklenen Çıkış:", true_output)
print("Model Çıkışı:", predicted_output)
print("F1 Skoru:", f1)


Giriş: 09:00 ile 10:00 arasında toplantı yapacağım, 12:00 ile 13:00 arasında öğle yemeği yapacağım, 16:00 ile 17:00 arasında arkadaş buluşması yapacağım, 17:00 ile 19:00 arasında yemek randevusu yapacağım, 21:00 ile 22:00 arasında kitap okuma yapacağım
Beklenen Çıkış: 09:00 - 10:00 toplantı, 12:00 - 13:00 öğle yemeği, 16:00 - 17:00 arkadaş buluşması, 17:00 - 19:00 yemek randevusu, 21:00 - 22:00 kitap okuma
Model Çıkışı: 09:00 - 10:00 toplantı  12:00 - 13:00 öğle yemeği  16:00 - 17:00 arkadaş buluşması  17:00 - 19:00 yemek randevusu  21:00 - 22:00 kitap okuma
F1 Skoru: 0.8333333333333334


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import re

# Model ve tokenizer yükleme
model_save_path = "/content/drive/MyDrive/planlama/2trained_model"
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_save_path)

# Cihaz ayarları
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def format_output(output_text):
    # Tüm varyasyonları yakalamak için regex desenlerini genişletme
    formatted_output = re.sub(r'(\d{1,2}):(\d{2}) (ile|ve|-) (\d{1,2}):(\d{2}) (arası|arasında)', r'\1:\2 - \4:\5', output_text, flags=re.IGNORECASE)
    formatted_output = re.sub(r'(\d{1,2}):(\d{2})-(\d{1,2}):(\d{2})', r'\1:\2 - \3:\4', formatted_output)  # Zaman aralıklarını düzeltme
    formatted_output = re.sub(r'(\d{1,2}) (ile|ve|-) (\d{1,2}) (arası|arasında)', r'\1:00 - \3:00', formatted_output, flags=re.IGNORECASE)

    # Gereksiz kelimeleri kaldırma
    formatted_output = re.sub(r'\b(yapacağım|yapacagim|var|olacak|yapmak)\b', '', formatted_output, flags=re.IGNORECASE)

    # Noktalama işaretlerini düzenleme
    formatted_output = formatted_output.replace(',', ' ').replace('  ', ' ').strip()

    # Sonundaki gereksiz karakterleri kaldırma
    formatted_output = re.sub(r'0+$', '', formatted_output)

    # Cümlenin sonuna noktaları ekleyerek düzenleme
    if formatted_output and formatted_output[-1] != '.':
        formatted_output += '.'

    return formatted_output.strip()

# Modelden çıktı üretme fonksiyonu
def generate_output(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True, padding="max_length")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return format_output(output_text.strip())

# Test fonksiyonu
def test_model():
    input_text = input("Giriş: ")
    predicted_output = generate_output(input_text)
    print("Model Çıkışı:", predicted_output)

# Modeli test etme
test_model()


Giriş:  9:00 ile 10:00 arasi dersim var. 15:00 ile 17:00 arasi yemek yiyecegim. 22 ile 23 arası film izleyecegim
Model Çıkışı: 9:00 - 10:00 dersim . 15:00 - 17:00 yemek yiyecegim. 22:00 - 23:00 film izleyecegim.


In [ ]:
!pip install flask flask-cors transformers torch pyngrok


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install pyngrok
!ngrok authtoken 2hFh039c1mHorUiI6urdqACI8M9_5y1JwajcHDmyudmRduovu
!pip install flask-cors

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Ngrok auth token'ınızı buraya ekleyin
ngrok.set_auth_token("2hFh039c1mHorUiI6urdqACI8M9_5y1JwajcHDmyudmRduovu")


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import re
from concurrent.futures import ThreadPoolExecutor
from pyngrok import ngrok
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
app = Flask(__name__)
CORS(app)

# Model ve tokenizer yükleme
model_save_path = "/content/drive/MyDrive/planlama/2trained_model"
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_save_path)

# Cihaz ayarları
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def format_output(output_text):
    # İstenmeyen işaretleri kaldır
    output_text = output_text.replace('------', '')

    formatted_output = re.sub(r'(\d{1,2}):(\d{2}) (ile|ve|-) (\d{1,2}):(\d{2}) (arası|arasında|arasinda|arasi)', r'\1:\2 - \4:\5', output_text, flags=re.IGNORECASE)
    formatted_output = re.sub(r'(\d{1,2}):(\d{2})-(\d{1,2}):(\d{2})', r'\1:\2 - \3:\4', formatted_output)  # Zaman aralıklarını düzeltme
    formatted_output = re.sub(r'(\d{1,2}) (ile|ve|-) (\d{1,2}) (arası|arasında)', r'\1:00 - \3:00', formatted_output, flags=re.IGNORECASE)
    formatted_output = re.sub(r'\b(yapacağım|yapacagim|var|olacak|yapmak|gidecegim|izleyecegim|oturacagım|icecegim|edecegim|yiyecegim|uygulayacagim|uyuyacagim|kalkacagim|calisacagim)\b', '', formatted_output, flags=re.IGNORECASE)
    formatted_output = formatted_output.replace(',', ' ').replace('  ', ' ').strip()
    formatted_output = re.sub(r'0+$', '', formatted_output)
    if formatted_output and formatted_output[-1] != '.':
        formatted_output += '.'
    return formatted_output.strip()




# Modelden çıktı üretme fonksiyonu
def generate_output(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True, padding="max_length")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return format_output(output_text.strip())

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    input_text = data['input_text']
    output_text = generate_output(input_text)
    return jsonify({'output_text': output_text})

def start_ngrok():
    url = ngrok.connect(5000)
    print(f'Ngrok Tunnel URL: {url}')

if __name__ == '__main__':
    with ThreadPoolExecutor() as executor:
        executor.submit(start_ngrok)
        app.run(port=5000)


Mounted at /content/drive
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Ngrok Tunnel URL: NgrokTunnel: "https://573a-35-185-231-208.ngrok-free.app" -> "http://localhost:5000"


INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:25:31] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:25:49] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:26:17] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:26:28] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:26:56] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:29:47] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:30:05] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:30:26] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:30:44] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:31:07] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:49:11] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 10:49:50] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:12